In [3]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import math
import random
from collections import defaultdict
import keras
from models import C3D
from keras import backend as K

In [8]:
class dataGenerator(keras.utils.Sequence):

    def __init__(self, filepath, batch_size, ffpath, segments = 16, test=False):
        self.filenames = list()
        self.labels = list()
        self.batch_size = batch_size
        self.filepath = filepath
        self.ffpath = ffpath
        
        with open(self.filepath,"r") as f:
            for line in f.readlines():
                arr = line.split(" ")
                self.filenames.append(arr[0])
                self.labels.append(int(arr[1].strip()))
        
    def __len__(self):
        return len(self.filenames)//self.batch_size

    def __getitem__(self, idx):
        batch_x = self.filenames[idx * self.batch_size:min((idx + 1) * self.batch_size, len(self.filenames))]
        batch_y = self.labels[idx * self.batch_size:min((idx + 1) * self.batch_size, len(self.filenames))]

        Xframes = list()
        Y = list()
        for index,each in enumerate(batch_x):
            infopath = os.path.join(self.ffpath,each,"info.txt")
            imgpath = os.path.join(self.ffpath,each,"frames")
            f = open(infopath,"r")
            total_frames = int(f.readlines()[0].strip().split(':')[1])
            f.close()
            idxs = sorted(np.random.randint(0, total_frames, 16))
            Xframes = self.getFrames(idxs, imgpath)
            Y.append(batch_y[index])

        finalX, finalY = Xframes, Y
        return (finalX,finalY)
    
    def one_hot_encode(self,data, classes = 101):
        """
        :param data: data to be one hot encoded
        :return: np array with one hot encoding
        """
        labels = np.zeros((data.size, classes))
        labels[np.arange(data.size), data - 1] = 1
        return labels
    
    def readImg(self,path):
        img = cv2.imread(path)
        grayimg = cv2.cvtColor(img,cv2.COLOR_BAYER_BG2RGB)
        return grayimg
    
    def getFrames(self,idxs, imgpath):
        stack = list()
        for i in idxs:
            framename = "frame_"+str(i)+".jpg"
            stack.append(self.readImg(os.path.join(imgpath,framename)))
        return np.array(stack)

In [9]:
# folderpath = "../ucfTrainTestlist"
# datapath = "../../UCF-101"
# filename = "trainlist01.txt"
# dg = dataGeneratorNew(datapath,os.path.join(folderpath,filename),batch_size = 128)



folderpath = "../ucfTrainTestlist"
filenameTrain = "custom2Train.txt"
filenameVal = "custom2Val.txt"

filepath = os.path.join(folderpath,filenameTrain)
ffpath = os.path.join("../../VidRecognizer_old/FramesFlows/custom2")
dgTrain = dataGenerator(filepath,16,ffpath)

filepath = os.path.join(folderpath,filenameVal)
ffpath = os.path.join("../../VidRecognizer_old/FramesFlows/custom2")
dgVal = dataGenerator(filepath,16,ffpath)

In [13]:
#Create and Compile model
K.clear_session()
model = C3D()
model.compile(optimizer= keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
np.random.seed(0)
#Fit generator
history = model.fit_generator(dgTrain, epochs = 1, validation_data = dgVal)

Epoch 1/1


FileNotFoundError: [Errno 2] No such file or directory: '../../VidRecognizer_old/FramesFlows/custom2/BenchPress/v_BenchPress_g10_c04.avi/info.txt'